# 네이버 영화평 감성분석

In [1]:
!pip install Konlpy > /dev/null #설치 과정 출력 안 되게.

In [2]:
import numpy as np
import pandas as pd

In [4]:
train_df=pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt',sep='\t') 
test_df=pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt',sep='\t')

In [5]:
print(train_df.shape, test_df.shape)
train_df.head(3)

(150000, 3) (50000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


 1. 데이터 전처리
 - train dataset에 대해.

In [6]:
# null 데이터 확인
train_df.isna().sum()

id          0
document    5
label       0
dtype: int64

In [9]:
# null 데이터 제거
train_df.dropna(how='any',inplace=True)
train_df.shape

(149995, 3)

In [10]:
#중복데이터 확인
train_df.document.nunique()

146182

In [12]:
#중복데이터 제거
train_df.drop_duplicates(subset=['document'],inplace=True)
train_df.shape

(146182, 3)

In [13]:
#긍정(1)/부정(0) 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

- test data에 대해.

In [14]:
# null 데이터 확인
test_df.isna().sum()

id          0
document    3
label       0
dtype: int64

In [15]:
# null 데이터 제거
test_df.dropna(how='any',inplace=True)
test_df.shape

(49997, 3)

In [19]:
#중복데이터 확인
test_df.document.nunique()

49157

In [20]:
#중복데이터 제거
test_df.drop_duplicates(subset=['document'],inplace=True)
test_df.shape

(49157, 3)

In [18]:
#긍정(1)/부정(0) 분포
test_df.label.value_counts()

1    24711
0    24446
Name: label, dtype: int64

2. 텍스트 전처리

In [21]:
#한글 이외 문자는 공백처리하고 strip
train_df.document = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ').str.strip()
train_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [22]:
#''만 남은 데이터 - np.nan으로 대체한 후 제거
train_df.document.replace('',np.nan,inplace=True)

In [24]:
train_df.document.isna().sum()

789

In [25]:
train_df.dropna(how='any',inplace=True)
train_df.shape

(145393, 3)

In [27]:
#한글 이외 문자는 공백처리하고 strip
test_df.document = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ').str.strip()
test_df.head()
#''만 남은 데이터 - np.nan으로 대체한 후 제거
test_df.document.replace('',np.nan,inplace=True)
test_df.document.isna().sum()
test_df.dropna(how='any',inplace=True)
test_df.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


(48852, 3)

In [28]:
#전처리 끝난 데이터 저장
train_df.to_csv('naver_movie_train_전처리완료.tsv',sep='\t',index=False)
test_df.to_csv('naver_movie_test_전처리완료.tsv',sep='\t',index=False)

3. 한글 처리

In [32]:
from konlpy.tag import Okt
okt=Okt()


In [33]:
text='교도소 이야기구먼 솔직히 재미는 없다평점 조정'
okt.morphs(text)

['교도소', '이야기', '구먼', '솔직히', '재미', '는', '없다', '평점', '조정']

In [34]:
okt.morphs(text,stem=True)

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [36]:
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을','ㅋㅋ','ㅠㅠ','ㅎㅎ']
' '.join([word for word in okt.morphs(text, stem=True) if word not in stopwords])
#리스트를 stem하고, 그걸 또 다시 벡터라이즈로 보냄. 
#벡터라이즈에는 리스트 못들어가고 텍스트로만 들어가야 돼서 
#이렇게 한번 더 작업해주는 것.

'교도소 이야기 구먼 솔직하다 재미 없다 평점 조정'

In [37]:
from tqdm.notebook import tqdm
X_train=[]
for sentence in tqdm(train_df.document):
    morphs = okt.morphs(sentence, stem=True)
    tmp_str = ' '.join([word for word in morphs if word not in stopwords])
    X_train.append(tmp_str)

  0%|          | 0/145393 [00:00<?, ?it/s]

In [38]:
X_test=[]
for sentence in tqdm(test_df.document):
    morphs = okt.morphs(sentence, stem=True)
    tmp_str = ' '.join([word for word in morphs if word not in stopwords])
    X_test.append(tmp_str)

  0%|          | 0/48852 [00:00<?, ?it/s]

In [42]:
y_train=train_df.label.values
y_test=test_df.label.values

### 4.피쳐 변환, 모델 학습/평가

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [43]:
cvect=CountVectorizer()
lr=LogisticRegression(random_state=2022)
pipeline=Pipeline([('CVECT',cvect),('LR',lr)])
%time pipeline.fit(X_train,y_train)


CPU times: user 8.33 s, sys: 14.7 s, total: 23 s
Wall time: 7.03 s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline(steps=[('CVECT', CountVectorizer()),
                ('LR', LogisticRegression(random_state=2022))])

In [45]:
with open('1.txt','w',encoding='UTF-8') as f:
    for i in X_test:
        f.write(i+'\n')

In [46]:
with open('2.txt','w',encoding='UTF-8') as f:
    for i in X_train:
        f.write(i+'\n')

In [47]:
pipeline.score(X_test,y_test)

0.8273356259723246

5. 실제 데이터 테스트

In [56]:
review1=' 영상미는 좋아요 마법액션이랑.근데. .. 흠 ... 너무 뻔한 이야기전개를 너무 뻔하게 만들었어요 ... 슬퍼... 해리포터 재탕할래여...'
review2=' 재미없는 시리즈 신작. 너무 분위기 우중충하고 전작들만큼 재미도 유머스러움도 없고 스토리도 별진전없이 허무하게 끝'
review3 = '모든 국민이 봤으면 하는 영화입니다.'
review4 = '생각보다 지루하고 별로였네요... 보면서 좀 졸았습니다.'

In [57]:
#전처리
import re
review1=re.sub('[^가-힣]',' ',review1)
review2=re.sub('[^가-힣]',' ',review2)
review3=re.sub('[^가-힣]',' ',review3)
review4=re.sub('[^가-힣]',' ',review4)

In [58]:
morphs = okt.morphs(review1, stem=True)
review1 = ' '.join([word for word in morphs if word not in stopwords])
morphs = okt.morphs(review2, stem=True)
review2 = ' '.join([word for word in morphs if word not in stopwords])
morphs = okt.morphs(review3, stem=True)
review3 = ' '.join([word for word in morphs if word not in stopwords])
morphs = okt.morphs(review4, stem=True)
review4 = ' '.join([word for word in morphs if word not in stopwords])

In [59]:
pipeline.predict([review1,review2,review3,review4])

array([1, 0, 1, 0])

6. 최적의 파라미터 찾기

In [60]:
from sklearn.model_selection import GridSearchCV
params={
    'CVECT__ngram_range':[(1,1),(1,2)],
    'CVECT__max_df':[0.95,0.98],
    'LR__C':[1.5]
}


In [62]:
grid_pipe=GridSearchCV(pipeline,params,scoring='accuracy',cv=3,n_jobs=-1)
%time grid_pipe.fit(X_train,y_train)

CPU times: user 36.4 s, sys: 41.2 s, total: 1min 17s
Wall time: 60 s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('CVECT', CountVectorizer()),
                                       ('LR',
                                        LogisticRegression(random_state=2022))]),
             n_jobs=-1,
             param_grid={'CVECT__max_df': [0.95, 0.98],
                         'CVECT__ngram_range': [(1, 1), (1, 2)],
                         'LR__C': [1.5]},
             scoring='accuracy')

In [63]:
grid_pipe.best_params_

{'CVECT__max_df': 0.95, 'CVECT__ngram_range': (1, 2), 'LR__C': 1.5}

In [64]:
grid_pipe.best_estimator_.score(X_test,y_test)

0.8479489069024809